In [6]:
!pip install pyspark


In [7]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
np.random.seed(10)
random_numbers = np.random.randint(0, 11, size=100)


conf = SparkConf().setAppName("FrequencyCalculation").setMaster("local[*]")
sc = SparkContext(conf=conf)
random_numbers_rdd = sc.parallelize(random_numbers)
number_count_rdd = random_numbers_rdd.map(lambda num: (num, 1))
number_frequencies = number_count_rdd.reduceByKey(lambda x, y: x + y)
result = number_frequencies.collect()

for num, frequency in sorted(result):
    print(f"Number: {num}, Frequency: {frequency}")
sc.stop()


Number: 0, Frequency: 12
Number: 1, Frequency: 11
Number: 2, Frequency: 8
Number: 3, Frequency: 6
Number: 4, Frequency: 8
Number: 5, Frequency: 5
Number: 6, Frequency: 11
Number: 7, Frequency: 5
Number: 8, Frequency: 14
Number: 9, Frequency: 12
Number: 10, Frequency: 8


We set the seed for the NumPy random number generator to 10, ensuring reproducibility of random numbers.

Using NumPy's randint function, we generate an array of 100 random integers between 0 and 10 (inclusive).

We configure Spark with an application name "FrequencyCalculation" and set it to run locally using all available cores.

We create an RDD (random_numbers_rdd) from the generated random numbers, map each number to a tuple of (number, 1) for counting (number_count_rdd), reduce by key to calculate the frequency of each number (number_frequencies), collect the results, and print them sorted by number. Finally, we stop the Spark context to release resources.

In [9]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("WordFrequency").setMaster("local[*]")  # Run Spark locally using all available cores
sc = SparkContext(conf=conf)
text8_rdd = sc.textFile("/content/drive/MyDrive/Datasets/text8.txt")
words_rdd = text8_rdd.flatMap(lambda line: line.split())
word_count_rdd = words_rdd.map(lambda word: (word, 1))
word_frequencies = word_count_rdd.reduceByKey(lambda x, y: x + y)
words_with_a = word_frequencies.filter(lambda pair: 'a' in pair[0])
result = words_with_a.collect()
for word, frequency in result:
    print(f"Word: {word}, Frequency: {frequency}")

sc.stop()


Streaming output truncated to the last 5000 lines.
Word: guacurian, Frequency: 1
Word: maratino, Frequency: 1
Word: reca, Frequency: 5
Word: meckna, Frequency: 1
Word: rockaway, Frequency: 2
Word: winlame, Frequency: 1
Word: sesodia, Frequency: 1
Word: maximi, Frequency: 1
Word: lionsbay, Frequency: 1
Word: zaobao, Frequency: 1
Word: thairath, Frequency: 1
Word: matichon, Frequency: 1
Word: eskalith, Frequency: 1
Word: anablepidae, Frequency: 1
Word: moscarda, Frequency: 2
Word: fontcuberta, Frequency: 1
Word: distionary, Frequency: 1
Word: lutra, Frequency: 1
Word: vinnacia, Frequency: 2
Word: cannesphotocall, Frequency: 1
Word: hospitalists, Frequency: 1
Word: amoron, Frequency: 1
Word: penaeaceae, Frequency: 2
Word: facioscapulohumeral, Frequency: 1
Word: reassuming, Frequency: 1
Word: matii, Frequency: 1
Word: bhandup, Frequency: 1
Word: kaminaljuy, Frequency: 1
Word: midwayislands, Frequency: 1
Word: ulaan, Frequency: 1
Word: capsian, Frequency: 2
Word: weihenmayer, Frequency: 1
W


The reduceByKey transformation aggregates the counts of each word by summing up the values associated with the same key (word), resulting in word_frequencies.
Filtering Words:

Using the filter transformation, words containing the letter 'a' are filtered from word_frequencies, resulting in words_with_a.

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, corr
spark = SparkSession.builder \
    .appName("IrisDataFrame") \
    .getOrCreate()

iris_df = spark.read.json("/content/drive/MyDrive/Datasets/iris.json")

correlation = iris_df.select(corr("petalLength", "petalWidth")).collect()[0][0]
print("Pearson Correlation between petalLength and petalWidth:", correlation)
filtered_df = iris_df.filter(col("petalLength") >= 1.4).select("sepalLength", "sepalWidth", "species")
filtered_df.show()
spark.stop()


Pearson Correlation between petalLength and petalWidth: 0.962865431402796
+-----------+----------+-------+
|sepalLength|sepalWidth|species|
+-----------+----------+-------+
|        5.1|       3.5| setosa|
|        4.9|       3.0| setosa|
|        4.6|       3.1| setosa|
|        5.0|       3.6| setosa|
|        5.4|       3.9| setosa|
|        4.6|       3.4| setosa|
|        5.0|       3.4| setosa|
|        4.4|       2.9| setosa|
|        4.9|       3.1| setosa|
|        5.4|       3.7| setosa|
|        4.8|       3.4| setosa|
|        4.8|       3.0| setosa|
|        5.7|       4.4| setosa|
|        5.1|       3.5| setosa|
|        5.7|       3.8| setosa|
|        5.1|       3.8| setosa|
|        5.4|       3.4| setosa|
|        5.1|       3.7| setosa|
|        5.1|       3.3| setosa|
|        4.8|       3.4| setosa|
+-----------+----------+-------+
only showing top 20 rows



The corr() function from pyspark.sql.functions is used to calculate the Pearson correlation between the columns "petalLength" and "petalWidth" in the iris_df DataFrame. The result is collected using collect() and indexed to retrieve the correlation value.